In [ ]:
%load_ext watermark


In [ ]:
import io
import os

from IPython.display import display
import matplotlib.pyplot as plt
import pymupdf
import seaborn as sns
from teeplot import teeplot as tp

import pylib  # noqa: F401


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = os.environ.get("NOTEBOOK_NAME", None)
teeplot_subdir


## Load PDFs


In [ ]:
positions_path = "assets/wse-spatial-phylo-positions.pdf"
template_path = "assets/wse-spatial-phylo-template.pdf"

positions_doc = pymupdf.open(positions_path)
print(f"Positions: {len(positions_doc)} page(s), size {positions_doc[0].rect}")

template_doc = pymupdf.open(template_path)
print(f"Template:  {len(template_doc)} page(s), size {template_doc[0].rect}")


## Find Colored Rectangle Positions


In [ ]:
target_colors = {
    "accede": "#ACCEDE",
    "beefed": "#BEEFED",
    "c0ffee": "#C0FFEE",
    "decade": "#DECADE",
    "efface": "#EFFACE",
}


def hex_to_rgb_float(hex_color):
    h = hex_color.lstrip("#")
    return tuple(int(h[i : i + 2], 16) / 255.0 for i in (0, 2, 4))


def find_rects_by_color(page, hex_color, tol=2 / 255):
    target = hex_to_rgb_float(hex_color)
    rects = []
    for path in page.get_drawings():
        fill = path.get("fill")
        if fill is None or len(fill) != 3:
            continue
        if all(abs(fill[i] - target[i]) < tol for i in range(3)):
            rects.append(path["rect"])
    return rects


positions_page = positions_doc[0]
color_rects = {}
for name, hex_color in target_colors.items():
    rects = find_rects_by_color(positions_page, hex_color)
    color_rects[name] = rects
    for r in rects:
        print(f"  {name} ({hex_color}): {r}")

positions_doc.close()


## Create Example Plots


In [ ]:
df = sns.load_dataset("mpg")


def make_plot_pdf(plot_func):
    """Call plot_func to create a figure, return as a pymupdf Document."""
    fig = plot_func()
    fig.patch.set_facecolor("none")
    fig.patch.set_edgecolor("none")
    for ax in fig.axes:
        ax.set_facecolor("none")
        legend = ax.get_legend()
        if legend is not None:
            legend.get_frame().set_facecolor("none")
            legend.get_frame().set_edgecolor("none")
    buf = io.BytesIO()
    fig.savefig(
        buf, format="pdf", bbox_inches="tight",
        facecolor="none", edgecolor="none",
    )
    plt.close(fig)
    buf.seek(0)
    return pymupdf.open("pdf", buf.read())


def plot_scatter():
    fig, ax = plt.subplots(figsize=(6, 4))
    for origin in df["origin"].unique():
        sub = df[df["origin"] == origin]
        ax.scatter(sub["horsepower"], sub["mpg"], label=origin, alpha=0.6)
    ax.set_xlabel("Horsepower")
    ax.set_ylabel("MPG")
    ax.set_title("Horsepower vs MPG")
    ax.legend()
    return fig


def plot_hist():
    fig, ax = plt.subplots(figsize=(6, 4))
    for origin in df["origin"].unique():
        ax.hist(df[df["origin"] == origin]["mpg"], alpha=0.5, label=origin)
    ax.set_xlabel("MPG")
    ax.set_ylabel("Count")
    ax.set_title("MPG Distribution")
    ax.legend()
    return fig


def plot_heatmap():
    fig, ax = plt.subplots(figsize=(6, 4))
    corr = df[["mpg", "displacement", "horsepower", "weight"]].corr()
    sns.heatmap(corr, annot=True, cmap="coolwarm", ax=ax)
    ax.set_title("Correlation Matrix")
    return fig


def plot_box():
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.boxplot(data=df, x="origin", y="mpg", ax=ax)
    ax.set_title("MPG by Origin")
    return fig


def plot_line():
    fig, ax = plt.subplots(figsize=(6, 4))
    yearly = df.groupby(["model_year", "origin"])["mpg"].mean().reset_index()
    for origin in yearly["origin"].unique():
        sub = yearly[yearly["origin"] == origin]
        ax.plot(sub["model_year"], sub["mpg"], marker="o", label=origin)
    ax.set_xlabel("Model Year")
    ax.set_ylabel("Mean MPG")
    ax.set_title("MPG Trend Over Time")
    ax.legend()
    return fig


plot_funcs = {
    "accede": plot_scatter,
    "beefed": plot_hist,
    "c0ffee": plot_heatmap,
    "decade": plot_box,
    "efface": plot_line,
}

plot_docs = {
    name: make_plot_pdf(func) for name, func in plot_funcs.items()
}
print("Generated vector plots:", list(plot_docs.keys()))


## Overlay Plots onto Template


In [ ]:
page = template_doc[0]

for name, rects in color_rects.items():
    src_doc = plot_docs[name]
    for rect in rects:
        page.show_pdf_page(rect, src_doc, 0)
        print(f"  Inserted {name} at {rect}")

output_destination = "teeplots/2026-02-14-fill-template/"
os.makedirs(output_destination, exist_ok=True)
output_path = os.path.join(output_destination, "wse-spatial-phylo-filled.pdf")
template_doc.save(output_path, garbage=4, deflate=True)
template_doc.close()
for d in plot_docs.values():
    d.close()
print(f"\nSaved to {output_path}")
